In [60]:
import requests


In [61]:
# Eurostat API connect and query generation using the Query generator from Eurostat
eurostat_url_service = 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/'

queryEurostat = "for_area?groupedIndicators=1&filterNonGeo=1&precision=1&indic_fo=FOR&unit=THS_HA&time=1990&time=2000&time=2010&time=2020"
rest_petition = eurostat_url_service + queryEurostat
response = requests.get(rest_petition)

# Print the Query in the URL
rest_petition

'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/for_area?groupedIndicators=1&filterNonGeo=1&precision=1&indic_fo=FOR&unit=THS_HA&time=1990&time=2000&time=2010&time=2020'

In [62]:
# As we know that the answer will be in JSON format..

import json

jsonEurostat = json.loads(response.text)
jsonEurostat
# the API response looks like that, in this case in the initial properties, we can see the "label" 
# title declaring = 'Area of wooded land (source: FAO - FE)'
# notice also the update date in order to check that the service is constantly updated.

{'version': '2.0',
 'label': 'Area of wooded land (source: FAO - FE)',
 'href': 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/for_area?groupedIndicators=1&filterNonGeo=1&precision=1&indic_fo=FOR&unit=THS_HA&time=1990&time=2000&time=2010&time=2020',
 'source': 'Eurostat',
 'updated': '2021-03-12',
 'extension': {'datasetId': 'for_area',
  'lang': 'EN',
  'description': None,
  'subTitle': None},
 'class': 'dataset',
 'value': {'0': 3775.67,
  '1': 3838.14,
  '2': 3863.2,
  '3': 3899.15,
  '4': 2210.0,
  '5': 2111.65,
  '6': 2102.66,
  '7': 2187.91,
  '8': 677.4,
  '9': 667.3,
  '10': 689.87,
  '11': 689.3,
  '12': 3327.0,
  '13': 3375.0,
  '14': 3737.0,
  '15': 3893.0,
  '16': 1153.5,
  '17': 1196.18,
  '18': 1234.72,
  '19': 1269.11,
  '20': 2629.42,
  '21': 2637.29,
  '22': 2657.38,
  '23': 2677.09,
  '24': 11300.0,
  '25': 11354.0,
  '26': 11409.0,
  '27': 11419.0,
  '28': 531.44,
  '29': 571.6,
  '30': 586.49,
  '31': 628.44,
  '32': 2205.9,
  '33': 2238.89,
  '34': 2336

In [114]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="App_prueba")

# As we are looking to represent the data in a map we will have to use a geocoding system to
# get and recognize the country codes so we import geopy.geocoders and pycountry_convert

# By other hand we must know that Eurostat give the data by countries in order, 
# so as we asked for 4 different year data collections we first must "group" the records
# by country and year

countries = {} #for now we would work with the data as a JSON not as an array
# lets use the JSON working format to get to the values we are searching for,
# in this case we want to acces to "dimension/geo/category/label" in order to get the
# relation between the unique country name and the country codes(primary keys)
# We select also tha country index if the primary key is the same as the country index 
# we establish the relation
for label in jsonEurostat['dimension']['geo']['category']['label']:
    for index in jsonEurostat['dimension']['geo']['category']['index']:
        if label == index:
            countries [jsonEurostat['dimension']['geo']['category']['index'][index]] = jsonEurostat['dimension']['geo']['category']['label'][label].replace(" (until 1990 former territory of the FRG)","")
            break

values= [] #We will build an Array with the country names
# We start defining our output table, you can notice that we insert the geolocalitation and
# the geocoding values
def camposTabla(): 
    valor['Pais'] = countries[countryId]
    valor['Valor'] = jsonEurostat['value'][dataId]
#     valor['numYear'] = x
    valor['Year']= year
    valor['Code']= country_name_to_country_alpha2(countries[countryId], cn_name_format="default")
    valor['latitude']= loc.latitude
    valor['longitude']= loc.longitude
    values.append(valor)
# As we have 4 values for each country and in the JSPN the years have a ID between 0 and 3 
# 0:1990 , 1:2000, 2: 2010, 3:2020 we would need to group the values
year= 1990
x = 0
for dataId in jsonEurostat['value']:
    groupId = (int(int(dataId)/4))
    for countryId in countries:
        if str(groupId) == str(countryId):
            valor = {}
            loc =geolocator.geocode(countries[countryId])
            if x == 0:
                camposTabla()
                x+=1
                year = 2000
                break
            if x == 1:
                camposTabla()
                x+=1
                year = 2010
                break
            if x == 2:
                camposTabla()
                x+=1
                year = 2020
                break
            if x == 3:
                camposTabla()
                x=0
                year = 1990
                break

values
# we can see how we ordered the data


[{'Pais': 'Austria',
  'Valor': 3775.67,
  'Year': 1990,
  'Code': 'AT',
  'latitude': 47.2,
  'longitude': 13.2},
 {'Pais': 'Austria',
  'Valor': 3838.14,
  'Year': 2000,
  'Code': 'AT',
  'latitude': 47.2,
  'longitude': 13.2},
 {'Pais': 'Austria',
  'Valor': 3863.2,
  'Year': 2010,
  'Code': 'AT',
  'latitude': 47.2,
  'longitude': 13.2},
 {'Pais': 'Austria',
  'Valor': 3899.15,
  'Year': 2020,
  'Code': 'AT',
  'latitude': 47.2,
  'longitude': 13.2},
 {'Pais': 'Bosnia and Herzegovina',
  'Valor': 2210.0,
  'Year': 1990,
  'Code': 'BA',
  'latitude': 44.3053476,
  'longitude': 17.5961467},
 {'Pais': 'Bosnia and Herzegovina',
  'Valor': 2111.65,
  'Year': 2000,
  'Code': 'BA',
  'latitude': 44.3053476,
  'longitude': 17.5961467},
 {'Pais': 'Bosnia and Herzegovina',
  'Valor': 2102.66,
  'Year': 2010,
  'Code': 'BA',
  'latitude': 44.3053476,
  'longitude': 17.5961467},
 {'Pais': 'Bosnia and Herzegovina',
  'Valor': 2187.91,
  'Year': 2020,
  'Code': 'BA',
  'latitude': 44.3053476,
  

In [115]:

import pandas as pd
# pd.set_option('display.max_rows', 500)
df = pd.DataFrame(values)
year_1990=df["Year"]==1990
filtro_year_1990=df[year_1990]
year_2000=df["Year"]==2000
filtro_year_2000=df[year_2000]
year_2010=df["Year"]==2010
filtro_year_2010=df[year_2010]
year_2020=df["Year"]==2020
filtro_year_2020=df[year_2020]

# We can calculate the mean "green" surface
df2 = df.groupby(["Pais","Code","latitude","longitude"]).Valor.mean()
df2 = df2.to_frame()

df



,Pais,Valor,Year,Code,latitude,longitude
0,Austria,3775.67,1990,AT,47.200000,13.200000
1,Austria,3838.14,2000,AT,47.200000,13.200000
2,Austria,3863.20,2010,AT,47.200000,13.200000
3,Austria,3899.15,2020,AT,47.200000,13.200000
4,Bosnia and Herzegovina,2210.00,1990,BA,44.305348,17.596147
5,Bosnia and Herzegovina,2111.65,2000,BA,44.305348,17.596147
6,Bosnia and Herzegovina,2102.66,2010,BA,44.305348,17.596147
7,Bosnia and Herzegovina,2187.91,2020,BA,44.305348,17.596147
8,Belgium,677.40,1990,BE,50.640281,4.666715
9,Belgium,667.30,2000,BE,50.640281,4.666715


In [116]:
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
import branca.colormap as cm
#empty map
linear = cm.LinearColormap(["green", "yellow", "red"], vmin=100, vmax=100000)

world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(df)):
        lat = df.iloc[i]['latitude']
        long = df.iloc[i]['longitude']
        radius=10
        popup_text = """<h4>Country Name :</h4> {}<br>
                    <h4>Area of Wooded Land :</h4> {}<br>
                    <h4>Year:</h4> {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['Pais'],
                                   df.iloc[i]['Valor'],
                                       df.iloc[i]['Year']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map


In [117]:
import geopip

geopip.search(lng=4.910248, lat=50.850981)

{'FIPS': 'BE',
 'ISO2': 'BE',
 'ISO3': 'BEL',
 'UN': 56,
 'NAME': 'Belgium',
 'AREA': 0,
 'POP2005': 10398049,
 'REGION': 150,
 'SUBREGION': 155,
 'LON': 4.664,
 'LAT': 50.643}

In [118]:
from shapely.geometry import Point

# combine lat and lon column to a shapely Point() object
df['geometry'] = df.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [119]:
import geopandas
import fiona
df = geopandas.GeoDataFrame(df,geometry='geometry') 
# df.to_json(r'C:\Users\gasca\Desktop\Export_geometry.json')
# df.to_json(r'C:\Users\gasca\Desktop\Export_geometry.json')
df
# df.to_file(r'C:\Users\gasca\Desktop\Export_geometry.geojson', driver='GeoJSON')  

,Pais,Valor,Year,Code,latitude,longitude,geometry
0,Austria,3775.67,1990,AT,47.200000,13.200000,POINT (13.20000 47.20000)
1,Austria,3838.14,2000,AT,47.200000,13.200000,POINT (13.20000 47.20000)
2,Austria,3863.20,2010,AT,47.200000,13.200000,POINT (13.20000 47.20000)
3,Austria,3899.15,2020,AT,47.200000,13.200000,POINT (13.20000 47.20000)
4,Bosnia and Herzegovina,2210.00,1990,BA,44.305348,17.596147,POINT (17.59615 44.30535)
5,Bosnia and Herzegovina,2111.65,2000,BA,44.305348,17.596147,POINT (17.59615 44.30535)
6,Bosnia and Herzegovina,2102.66,2010,BA,44.305348,17.596147,POINT (17.59615 44.30535)
7,Bosnia and Herzegovina,2187.91,2020,BA,44.305348,17.596147,POINT (17.59615 44.30535)
8,Belgium,677.40,1990,BE,50.640281,4.666715,POINT (4.66671 50.64028)
9,Belgium,667.30,2000,BE,50.640281,4.666715,POINT (4.66671 50.64028)
